# Example of creating a package

In [ ]:
# install Python & create venv
# cd to correct dir
./configure --prefix=/home/ec2-user/SageMaker/tmp/python/
make
make install

# now new python is in 
/home/ec2-user/SageMaker/tmp/python/

# create a new venv

v="/dev/shm/venv/"
# /home/ec2-user/SageMaker/tmp/python/bin/python3 -m venv $v --clear # broken as python 3.7 needs libffi-dev 
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/bin/python3 -m venv $v --clear
source $v"bin/activate"
pip install --no-index ~/SageMaker/tmp/pip-20.0.2-py2.py3-none-any.whl
pip install --no-index ~/SageMaker/tmp/setuptools-46.1.1-py3-none-any.whl


# stash away the venv
cp -rf /dev/shm/venv /dev/shm/venv0

# each time using the /dev/shm/venv
rm -rf /dev/shm/venv
cp -rf /dev/shm/venv0 /dev/shm/venv

### create a script to init some files

In [1]:
d0 = '/dev/shm/test_package/'
d = d0 + 'fruit/'

In [2]:
%%writefile /dev/shm/init.sh
#!/bin/bash
d0="/dev/shm/test_package/"
d=$d0"fruit/"

[[ -v V ]] && echo -e "init dir\n"
rm -rf $d0

# create some dir structure
mkdir -p $d0"veg"
mkdir -p $d
mkdir -p $d"citrus"
mkdir -p $d"citrus/orange"
mkdir -p $d"citrus/orange/common"
mkdir -p $d"citrus/orange/navel"
# mkdir -p $d"citrus/orange/valencia"
mkdir -p $d"citrus/mandarin"
# mkdir -p $d"citrus\grapefruit"
mkdir -p $d"berrie"
# mkdir -p $d"stone"

touch $d0"empty.py"
echo -e "123 fruit" > $d"fruit.txt"
echo -e "123 orange text" > $d"citrus/orange/orange.txt"

echo -e "123 veg" > $d0"veg/veg.txt"


# if Verbsoe, print the tree
[[ -v V ]] && find ${d0::-1} | sed -e "s/[^-][^\/]*\// |/g" -e "s/|\([^ ]\)/|-\1/"
# ls -R /tmp/test_package | grep ":$" | sed -e 's/:$//' -e 's/[^-][^\/]*\//--/g' -e 's/^/   /' -e 's/-/|/'

Overwriting /dev/shm/init.sh


In [3]:
!chmod +x /dev/shm/init.sh

In [4]:
!V=1 /dev/shm/init.sh

init dir

 | |-test_package
 | | |-empty.py
 | | |-fruit
 | | | |-fruit.txt
 | | | |-berrie
 | | | |-citrus
 | | | | |-mandarin
 | | | | |-orange
 | | | | | |-orange.txt
 | | | | | |-navel
 | | | | | |-common
 | | |-veg
 | | | |-veg.txt


### setuptools need \_\_init\_\_.py to find packages

In [5]:
%%bash

d0="/dev/shm/test_package/"
d=$d0"fruit/"


function f() {
    cd $d0
    printf "$1 --- Packages: "
    python3 -c "print(__import__('setuptools').find_packages())"
} 


# without the __init__.py, the setuptools can't find any package
find $d0 -name __init__.py -exec rm -f {} + # remove all the __init__.py
f "No __init__.py";


# create __init__.py at the top level
touch $d"__init__.py"
f "\nfruit has __init__.py"


# # create __init__.py recursively 
find ${d0::-1} -type d -exec touch {}/__init__.py \;
f "\nAll have __init__.py"


# tree
echo
find ${d0::-1} | sed -e "s/[^-][^\/]*\// |/g" -e "s/|\([^ ]\)/|-\1/"


# clean up
echo -e "\nClean up and init again"
PrintTree=1 /dev/shm/init.sh


No __init__.py --- Packages: []

fruit has __init__.py --- Packages: ['fruit']

All have __init__.py --- Packages: ['fruit', 'veg', 'fruit.berrie', 'fruit.citrus', 'fruit.citrus.mandarin', 'fruit.citrus.orange', 'fruit.citrus.orange.navel', 'fruit.citrus.orange.common']

 | |-test_package
 | | |-__init__.py
 | | |-empty.py
 | | |-fruit
 | | | |-__init__.py
 | | | |-fruit.txt
 | | | |-berrie
 | | | | |-__init__.py
 | | | |-citrus
 | | | | |-__init__.py
 | | | | |-mandarin
 | | | | | |-__init__.py
 | | | | |-orange
 | | | | | |-__init__.py
 | | | | | |-orange.txt
 | | | | | |-navel
 | | | | | | |-__init__.py
 | | | | | |-common
 | | | | | | |-__init__.py
 | | |-veg
 | | | |-__init__.py
 | | | |-veg.txt

Clean up and init again


## Make a virtual env

### Script to "clone" a new env

In [6]:
%%writefile /dev/shm/env.sh
#!/bin/bash
[[ -v V ]] && echo "actiave venv"

# make /dev/shm/venv/ is installed already
export v="/dev/shm/venv/"
rm -rf $v
cp -rf ${v::-1}0 $v
source $v"bin/activate"

[[ -v V ]] && echo $(which python && python --version)
[[ -v V ]] && echo $(which pip && pip --version) # should be 20+

Writing /dev/shm/env.sh


In [7]:
!chmod +x /dev/shm/env.sh

In [8]:
!V=1 /dev/shm/env.sh

actiave venv
Python 3.6.10 :: Anaconda, Inc.
/dev/shm/venv/bin/python
/dev/shm/venv/bin/pip pip 20.0.2 from /dev/shm/venv/lib/python3.6/site-packages/pip (python 3.6)


### Script to install setup.py

Before installing, clean up the /dev/shm/test_package
1. Clone a new venv
2. copy the $1 into setup.py
3. pip install
4. ls the install dir


In [9]:
%%writefile /dev/shm/install.sh
#!/bin/bash
source /dev/shm/env.sh # clone venv

# create the setup.py
cd /dev/shm/test_package/
[[ -v V ]] && echo -e "\nPWD: $(pwd) \n"
echo "$1" > setup.py

pip install --no-index --no-cache-dir --no-compile .

echo
ls /dev/shm/venv/lib/python3.6/site-packages/

echo
find /dev/shm/venv/lib/python3.6/site-packages/fruit | sed -e "s/[^-][^\/]*\// |/g" -e "s/|\([^ ]\)/|-\1/";


Writing /dev/shm/install.sh


In [10]:
!chmod +x /dev/shm/install.sh

In [11]:
!/dev/shm/init.sh
!V=1 /dev/shm/install.sh "__import__('setuptools').setup()"

actiave venv
Python 3.6.10 :: Anaconda, Inc.
/dev/shm/venv/bin/python
/dev/shm/venv/bin/pip pip 20.0.2 from /dev/shm/venv/lib/python3.6/site-packages/pip (python 3.6)

PWD: /dev/shm/test_package 

Processing /dev/shm/test_package
    Running setup.py install for UNKNOWN ... done

easy_install.py       pkg_resources  setuptools-46.1.1.dist-info
pip		      __pycache__    UNKNOWN-0.0.0-py3.6.egg-info
pip-20.0.2.dist-info  setuptools

find: `/dev/shm/venv/lib/python3.6/site-packages/fruit': No such file or directory


## Installing package 

### No package
No fruit is installed, as `packages=['']`

In [12]:
%%bash
f="from setuptools import setup, find_packages
setup(
    name='myPackage', # can be anything, space, hyphen, cases
    packages=[''],
)"

/dev/shm/init.sh
/dev/shm/install.sh "$f"

Processing /dev/shm/test_package
    Running setup.py install for myPackage: started
    Running setup.py install for myPackage: finished with status 'done'

easy_install.py
empty.py
myPackage-0.0.0-py3.6.egg-info
pip
pip-20.0.2.dist-info
pkg_resources
__pycache__
setuptools
setuptools-46.1.1.dist-info



find: `/dev/shm/venv/lib/python3.6/site-packages/fruit': No such file or directory


### No \_\_init\_\_.py
Also no fruit is installed

In [14]:
%%bash
f="from setuptools import setup, find_packages
setup(
    name='myPackage', # can be anything, space, hyphen, cases
    packages=[*find_packages()],
)"

/dev/shm/init.sh
/dev/shm/install.sh "$f"
# no fruit package installed since there is no __init__.py

Processing /dev/shm/test_package
    Running setup.py install for myPackage: started
    Running setup.py install for myPackage: finished with status 'done'

easy_install.py
myPackage-0.0.0-py3.6.egg-info
pip
pip-20.0.2.dist-info
pkg_resources
__pycache__
setuptools
setuptools-46.1.1.dist-info



find: `/dev/shm/venv/lib/python3.6/site-packages/fruit': No such file or directory


### \_\_init\_\_.py in some

In [15]:
%%bash
f="from setuptools import setup, find_packages
setup(
    name='myPackage', # can be anything, space, hyphen, cases
    packages=[*find_packages()],
)"

/dev/shm/init.sh
echo "print('__init__ from fruit')" > /dev/shm/test_package/fruit/__init__.py
echo "print('Fruit')" > /dev/shm/test_package/fruit/fruit.py


/dev/shm/install.sh "$f" # There is a fruit package

cd ~ && echo -e "\nshow some testing\n" && source /dev/shm/venv/bin/activate

python -c "import fruit" # should work
python -c "import fruit.fruit" # should work
python -c "import fruit.citrus" # should not work

Processing /dev/shm/test_package
    Running setup.py install for myPackage: started
    Running setup.py install for myPackage: finished with status 'done'

easy_install.py
fruit
myPackage-0.0.0-py3.6.egg-info
pip
pip-20.0.2.dist-info
pkg_resources
__pycache__
setuptools
setuptools-46.1.1.dist-info

 | | | | | |-fruit
 | | | | | | |-fruit.py
 | | | | | | |-__init__.py

show some testing

__init__ from fruit
__init__ from fruit
Fruit
__init__ from fruit


Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'fruit.citrus'


### \_\_inti\_\_.py in all

In [16]:
%%bash
f="from setuptools import setup, find_packages
setup(
    name='myPackage', # can be anything, space, hyphen, cases
    packages=[*find_packages()],
)"

/dev/shm/init.sh
find /dev/shm/test_package/ -type d -exec touch {}/__init__.py \; # insert __init__.py into all dirs
echo -e "print('__init__ from fruit')" > /dev/shm/test_package/fruit/__init__.py
echo -e "print('citrus')" > /dev/shm/test_package/fruit/citrus/__init__.py

/dev/shm/install.sh "$f"

cd ~ && echo -e "\nshow some testing\n" && source /dev/shm/venv/bin/activate

python -c "import fruit" # should work
python -c "import fruit.citrus" # should work too

deactivate

echo -e "\nstep out of the venv"
python -c "import fruit" # should not work

Processing /dev/shm/test_package
    Running setup.py install for myPackage: started
    Running setup.py install for myPackage: finished with status 'done'

easy_install.py
fruit
myPackage-0.0.0-py3.6.egg-info
pip
pip-20.0.2.dist-info
pkg_resources
__pycache__
setuptools
setuptools-46.1.1.dist-info
veg

 | | | | | |-fruit
 | | | | | | |-berrie
 | | | | | | | |-__init__.py
 | | | | | | |-__init__.py
 | | | | | | |-citrus
 | | | | | | | |-mandarin
 | | | | | | | | |-__init__.py
 | | | | | | | |-__init__.py
 | | | | | | | |-orange
 | | | | | | | | |-common
 | | | | | | | | | |-__init__.py
 | | | | | | | | |-__init__.py
 | | | | | | | | |-navel
 | | | | | | | | | |-__init__.py

show some testing

__init__ from fruit
__init__ from fruit
citrus

step out of the venv


Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'fruit'


In [17]:
%%bash
cd /dev/shm/test_package/

echo -e "cd into /dev/shm/test_package/"
python -c "import fruit" # should work
python -c "import fruit.citrus" # should work

cd into /dev/shm/test_package/
__init__ from fruit
__init__ from fruit
citrus


In [18]:
%%bash
cd /dev/shm/test_package/fruit/

echo -e "cd into /dev/shm/test_package/fruit\n"
ls
echo
python -c "import citrus" # should work
python -c "import fruit" # should not work


cd into /dev/shm/test_package/fruit

berrie
citrus
fruit.txt
__init__.py
__pycache__

citrus


Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'fruit'


### with MANIFEST
use `package=['fruit']` to include only fruit, no need for \_\_init\_\_.py

In [19]:
%%bash
/dev/shm/init.sh

f="from setuptools import setup, find_packages
setup(
    name='myPackage', # can be anything, space, hyphen, cases
    # package_data={ '': ['*'] },
    # package_dir={ 'fruit': 'fruit' },
    # packages=find_packages(),
    packages=['fruit'], # which dir to include
    include_package_data=True, # must have this
)"

# no need to have __init__.py

echo -e "global-include fruit *
global-exclude __pycache__*
global-exclude *.pyc
global-exclude *.pyo"> /dev/shm/test_package/MANIFEST.in

echo -e "print('mandarin is called')" > /dev/shm/test_package/fruit/citrus/mandarin/mandarin.py

/dev/shm/install.sh "$f"

echo -e "\nls the dir:\n"

cd ~ && echo -e "\nshow some testing\n" && source /dev/shm/venv/bin/activate


python -c "import fruit" # should work
python -c "import fruit.citrus" # should work
python -c "import fruit.citrus.orange" # should work
python -c "import fruit.citrus.mandarin.mandarin" # should work

# only dir with file will be included
python -c "import fruit.berrie" # should not work, because berrie has no file
python -c "import fruit.citrus.orange.naval" # should not work work

Processing /dev/shm/test_package
    Running setup.py install for myPackage: started
    Running setup.py install for myPackage: finished with status 'done'

easy_install.py
fruit
myPackage-0.0.0-py3.6.egg-info
pip
pip-20.0.2.dist-info
pkg_resources
__pycache__
setuptools
setuptools-46.1.1.dist-info

 | | | | | |-fruit
 | | | | | | |-citrus
 | | | | | | | |-mandarin
 | | | | | | | | |-mandarin.py
 | | | | | | | |-orange
 | | | | | | | | |-orange.txt
 | | | | | | |-fruit.txt

ls the dir:


show some testing

mandarin is called


Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'fruit.berrie'
Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'fruit.citrus.orange.naval'


### with MANIFEST
Specify `packages=['']` to include everything

In [20]:
%%bash
/dev/shm/init.sh

f="from setuptools import setup, find_packages
setup(
    name='my PackAge', # can be anything, space, hyphen, cases
    packages=[''], # everything
    include_package_data=True, # must have this
)"

echo -e "global-include *
global-exclude __pycache__*
global-exclude *.pyc
global-exclude *.pyo"> /dev/shm/test_package/MANIFEST.in

/dev/shm/install.sh "$f"

echo -e "\nls the dir"
ls /dev/shm/venv/lib/python3.6/site-packages/veg/

Processing /dev/shm/test_package
    Running setup.py install for my-PackAge: started
    Running setup.py install for my-PackAge: finished with status 'done'

easy_install.py
empty.py
fruit
MANIFEST.in
my_PackAge-0.0.0-py3.6.egg-info
my_PackAge.egg-info
pip
pip-20.0.2.dist-info
pip-delete-this-directory.txt
pip-egg-info
pkg_resources
__pycache__
setuptools
setuptools-46.1.1.dist-info
veg

 | | | | | |-fruit
 | | | | | | |-citrus
 | | | | | | | |-orange
 | | | | | | | | |-orange.txt
 | | | | | | |-fruit.txt

ls the dir
veg.txt


### with MANIFEST & exclude
.py seem cannot be excluded

In [21]:
%%bash
/dev/shm/init.sh

f="from setuptools import setup, find_packages
setup(
    name='my PackAge', # can be anything, space, hyphen, cases
    packages=[''], # everything
    include_package_data=True, # must have this
)"

echo -e "global-include *
global-exclude __pycache__*
global-exclude *.pyc
global-exclude *.pyo
global-exclude *.txt
exclude empty.py
"> /dev/shm/test_package/MANIFEST.in

# 
echo -e "print('mandarin is called')" > /dev/shm/test_package/fruit/citrus/mandarin/mandarin.py


/dev/shm/install.sh "$f"

echo -e "\nshow some testing\n"
cd ~
source /dev/shm/venv/bin/activate

python -c "import fruit" # should work
python -c "import fruit.citrus" # should work
python -c "import fruit.citrus.mandarin.mandarin" # should work


python -c "import fruit.citrus.orange" # should not work

Processing /dev/shm/test_package
    Running setup.py install for my-PackAge: started
    Running setup.py install for my-PackAge: finished with status 'done'

easy_install.py
empty.py
fruit
MANIFEST.in
my_PackAge-0.0.0-py3.6.egg-info
my_PackAge.egg-info
pip
pip-20.0.2.dist-info
pip-egg-info
pkg_resources
__pycache__
setuptools
setuptools-46.1.1.dist-info

 | | | | | |-fruit
 | | | | | | |-citrus
 | | | | | | | |-mandarin
 | | | | | | | | |-mandarin.py

show some testing

mandarin is called


Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'fruit.citrus.orange'


In [22]:
!find $d | sed -e "s/[^-][^\/]*\// |/g" -e "s/|\([^ ]\)/|-\1/"

 | | | |
 | | | |-fruit.txt
 | | | |-berrie
 | | | |-citrus
 | | | | |-mandarin
 | | | | | |-mandarin.py
 | | | | |-orange
 | | | | | |-orange.txt
 | | | | | |-navel
 | | | | | |-common


### Integrated test

In [23]:
%%writefile /dev/shm/test.sh
#!/bin/bash
d="/dev/shm/test_package/fruit/"

echo -e "global-include *"> /dev/shm/test_package/MANIFEST.in
/dev/shm/install.sh "__import__('setuptools').setup(name='A', packages=[''], include_package_data=True)"

cd ~ && echo -e "\nshow some testing\n" && source /dev/shm/venv/bin/activate

Writing /dev/shm/test.sh


In [24]:
!chmod +x /dev/shm/test.sh

### Simple structure

In [26]:
%%bash
source /dev/shm/init.sh

echo -e "print('citrus __init__')" > $d"citrus/__init__.py"
source /dev/shm/test.sh

python -c "import fruit.citrus"

Processing /dev/shm/test_package
    Running setup.py install for A: started
    Running setup.py install for A: finished with status 'done'

A-0.0.0-py3.6.egg-info
A.egg-info
easy_install.py
empty.py
fruit
MANIFEST.in
pip
pip-20.0.2.dist-info
pip-delete-this-directory.txt
pip-egg-info
pkg_resources
__pycache__
setuptools
setuptools-46.1.1.dist-info
veg

 | | | | | |-fruit
 | | | | | | |-citrus
 | | | | | | | |-__init__.py
 | | | | | | | |-orange
 | | | | | | | | |-orange.txt
 | | | | | | |-fruit.txt

show some testing

citrus __init__


### Running as normal script, without installing as package

In [28]:
%%bash
source /dev/shm/init.sh

echo -e "print('citrus __init__', __package__); import citrus.orange" > $d"citrus/__init__.py"
echo -e "print('orange __init__', __package__)" > $d"citrus/orange/__init__.py"

# source /dev/shm/test.sh
source /dev/shm/env.sh

cd $d
pwd
python -c "import citrus"

/dev/shm/test_package/fruit
citrus __init__ citrus
orange __init__ citrus.orange


In [29]:
%%bash
source /dev/shm/init.sh

echo -e "print('citrus __init__', __package__); from citrus import orange" > $d"citrus/__init__.py"
echo -e "print('orange __init__', __package__)" > $d"citrus/orange/__init__.py"

source /dev/shm/env.sh

cd $d
pwd
python -c "import citrus"

/dev/shm/test_package/fruit
citrus __init__ citrus
orange __init__ citrus.orange


### need to use relative import

In [32]:
%%bash
source /dev/shm/init.sh

echo -e "print('citrus __init__', __package__); import orange" > $d"citrus/__init__.py" # doesn't work, see cell below
echo -e "print('orange __init__', __package__)" > $d"citrus/orange/__init__.py"

source /dev/shm/env.sh

cd $d
pwd
python -c "import citrus"

/dev/shm/test_package/fruit
citrus __init__ citrus


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/dev/shm/test_package/fruit/citrus/__init__.py", line 1, in <module>
    print('citrus __init__', __package__); import orange
ModuleNotFoundError: No module named 'orange'


In [33]:
%%bash
source /dev/shm/init.sh

echo -e "print('citrus __init__', __package__); from . import orange" > $d"citrus/__init__.py" # work as it's relative import
echo -e "print('orange __init__', __package__)" > $d"citrus/orange/__init__.py"

source /dev/shm/env.sh

cd $d
pwd
python -c "import citrus"

/dev/shm/test_package/fruit
citrus __init__ citrus
orange __init__ citrus.orange


In [34]:
%%bash
source /dev/shm/init.sh

echo -e "print('citrus __init__', __package__); from . import orange" > $d"citrus/__init__.py" # work as it's relative import
echo -e "print('orange __init__', __package__); from .. import mandarin" > $d"citrus/orange/__init__.py" # 1 level up
echo -e "123 mandarin text" > $d"citrus/mandarin/mandarin.txt"
echo -e "import os
print()
print('mandarin __init__')
print('__package__', __package__)
print('os.getcwd()', os.getcwd())
print('__file__', __file__)
print('os.path.dirname', os.path.dirname(__file__))
print()
with open('citrus/mandarin/mandarin.txt') as f: print(f.read())
with open('citrus/orange/orange.txt') as f: print(f.read())" > $d"citrus/mandarin/__init__.py"


source /dev/shm/env.sh

cd $d
pwd
python -c "import citrus"

/dev/shm/test_package/fruit
citrus __init__ citrus
orange __init__ citrus.orange

mandarin __init__
__package__ citrus.mandarin
os.getcwd() /dev/shm/test_package/fruit
__file__ /dev/shm/test_package/fruit/citrus/mandarin/__init__.py
os.path.dirname /dev/shm/test_package/fruit/citrus/mandarin

123 mandarin text

123 orange text



### reading data file

In [35]:
%%bash
source /dev/shm/init.sh

echo -e "print('citrus __init__', __package__); from . import orange" > $d"citrus/__init__.py" # work as it's relative import
echo -e "print('orange __init__', __package__); from .. import mandarin" > $d"citrus/orange/__init__.py" # 1 level up
echo -e "123 mandarin text" > $d"citrus/mandarin/mandarin.txt"
echo -e "import os
print()
print('mandarin __init__')
print('__package__', __package__)
print('os.getcwd()', os.getcwd())
print('__file__', __file__)
print('os.path.dirname', os.path.dirname(__file__))

dirname = os.path.dirname(__file__)
path = os.path.join(dirname, '../..')
print('path', path)

print()
with open(path + '/citrus/mandarin/mandarin.txt') as f: print(f.read())
with open(path + '/citrus/orange/orange.txt') as f: print(f.read())" > $d"citrus/mandarin/__init__.py"


source /dev/shm/env.sh

cd $d
pwd
python -c "import citrus"

/dev/shm/test_package/fruit
citrus __init__ citrus
orange __init__ citrus.orange

mandarin __init__
__package__ citrus.mandarin
os.getcwd() /dev/shm/test_package/fruit
__file__ /dev/shm/test_package/fruit/citrus/mandarin/__init__.py
os.path.dirname /dev/shm/test_package/fruit/citrus/mandarin
path /dev/shm/test_package/fruit/citrus/mandarin/../..

123 mandarin text

123 orange text



### when built as package, data file can't be found in open(...)

In [36]:
%%bash
source /dev/shm/init.sh

echo -e "print('citrus __init__', __package__); from . import orange" > $d"citrus/__init__.py" # work as it's relative import
echo -e "print('orange __init__', __package__); from .. import mandarin" > $d"citrus/orange/__init__.py" # 1 level up
echo -e "123 mandarin text" > $d"citrus/mandarin/mandarin.txt"
echo -e "import os
print()
print('mandarin __init__')
print('__package__', __package__)
print('os.getcwd()', os.getcwd())
print('__file__', __file__)
print()
with open('citrus/mandarin/mandarin.txt') as f: print(f.read())
with open('citrus/orange/orange.txt') as f: print(f.read())" > $d"citrus/mandarin/__init__.py"


source /dev/shm/test.sh

pwd
python -c "import fruit.citrus"

Processing /dev/shm/test_package
    Running setup.py install for A: started
    Running setup.py install for A: finished with status 'done'

A-0.0.0-py3.6.egg-info
A.egg-info
easy_install.py
empty.py
fruit
MANIFEST.in
pip
pip-20.0.2.dist-info
pip-delete-this-directory.txt
pip-egg-info
pkg_resources
__pycache__
setuptools
setuptools-46.1.1.dist-info
veg

 | | | | | |-fruit
 | | | | | | |-citrus
 | | | | | | | |-mandarin
 | | | | | | | | |-mandarin.txt
 | | | | | | | | |-__init__.py
 | | | | | | | |-__init__.py
 | | | | | | | |-orange
 | | | | | | | | |-orange.txt
 | | | | | | | | |-__init__.py
 | | | | | | |-fruit.txt

show some testing

/home/ec2-user
citrus __init__ fruit.citrus
orange __init__ fruit.citrus.orange

mandarin __init__
__package__ fruit.citrus.mandarin
os.getcwd() /home/ec2-user
__file__ /dev/shm/venv/lib/python3.6/site-packages/fruit/citrus/mandarin/__init__.py



Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/dev/shm/venv/lib/python3.6/site-packages/fruit/citrus/__init__.py", line 1, in <module>
    print('citrus __init__', __package__); from . import orange
  File "/dev/shm/venv/lib/python3.6/site-packages/fruit/citrus/orange/__init__.py", line 1, in <module>
    print('orange __init__', __package__); from .. import mandarin
  File "/dev/shm/venv/lib/python3.6/site-packages/fruit/citrus/mandarin/__init__.py", line 8, in <module>
    with open('citrus/mandarin/mandarin.txt') as f: print(f.read())
FileNotFoundError: [Errno 2] No such file or directory: 'citrus/mandarin/mandarin.txt'


### Use __file__ & os.path should work

In [37]:
%%bash
source /dev/shm/init.sh

echo -e "print('citrus __init__', __package__); from . import orange; from . import c" > $d"citrus/__init__.py"
echo -e "print('c'); from . import c2" > $d"citrus/c.py"
echo -e "print('c2')" > $d"citrus/c2.py"


echo -e "print('orange __init__', __package__); from .. import mandarin" > $d"citrus/orange/__init__.py" # 1 level up
echo -e "123 mandarin text" > $d"citrus/mandarin/mandarin.txt"
echo -e "import os
print()
print('mandarin __init__')
print('__package__', __package__)
print('os.getcwd()', os.getcwd())
print('__file__', __file__)
print('os.path.dirname', os.path.dirname(__file__))

dirname = os.path.dirname(__file__)
path = os.path.join(dirname, '../..')
print('path', path)

print()
with open(path + '/citrus/mandarin/mandarin.txt') as f: print(f.read())
with open(path + '/citrus/orange/orange.txt') as f: print(f.read())" > $d"citrus/mandarin/__init__.py"


source /dev/shm/test.sh

pwd
python -c "import fruit.citrus"

Processing /dev/shm/test_package
    Running setup.py install for A: started
    Running setup.py install for A: finished with status 'done'

A-0.0.0-py3.6.egg-info
A.egg-info
easy_install.py
empty.py
fruit
MANIFEST.in
pip
pip-20.0.2.dist-info
pip-delete-this-directory.txt
pip-egg-info
pkg_resources
__pycache__
setuptools
setuptools-46.1.1.dist-info
veg

 | | | | | |-fruit
 | | | | | | |-citrus
 | | | | | | | |-mandarin
 | | | | | | | | |-mandarin.txt
 | | | | | | | | |-__init__.py
 | | | | | | | |-c2.py
 | | | | | | | |-__init__.py
 | | | | | | | |-orange
 | | | | | | | | |-orange.txt
 | | | | | | | | |-__init__.py
 | | | | | | | |-c.py
 | | | | | | |-fruit.txt

show some testing

/home/ec2-user
citrus __init__ fruit.citrus
orange __init__ fruit.citrus.orange

mandarin __init__
__package__ fruit.citrus.mandarin
os.getcwd() /home/ec2-user
__file__ /dev/shm/venv/lib/python3.6/site-packages/fruit/citrus/mandarin/__init__.py
os.path.dirname /dev/shm/venv/lib/python3.6/site-packages/fruit/